# Data Wrangling Challenge
### Pull and manipulate the API data

The point of this exercise is to try data enrichment with data from external APIs. We are going to take data about car crashes in Monroe County, Indiana from 2003 to 2015 and try to figure out the weather during the accident and how many bars there are in the area. We will work with two different APIs during this challenge:

- Foursquare API
- World Weather Online API

We will try to find correlations between the severity of crash and weather/number of bars in the area. To indicate the severity of a crash, we will use column `Injury Type`.

## Data

The data for this exercise can be found [here](https://drive.google.com/file/d/1_KF9oIJV8cB8i3ngA4JPOLWIE_ETE6CJ/view?usp=sharing).

Just run the cells below to get your data ready. Little help from us.


In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
from matplotlib import test
import json
from pandas import json_normalize


In [ ]:
data = pd.read_csv("monroe-county-crash-data2003-to-2015.csv", encoding="unicode_escape")
# ========================
# preparing data
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)
# creation of variable with lon and lat together
data['ll'] = data['Latitude'].astype(str) + ',' + data['Longitude'].astype(str)
data = data[data['ll'] != '0.0,0.0']
print(data.shape)
data.head(3)

In [103]:
type(data['ll'][0])

str

# Foursquare API

Foursquare API documentation is [here](https://developer.foursquare.com/)

1. Start a foursquare application and get your keys.
2. For each crash, create the function **get_venues** that will pull bars in the radius of 5km around the crash

#### example
`get_venues('48.146394, 17.107969')`

3. Find a relationship (if there is any) between number of bars in the area and severity of the crash.

HINTs: 
- check out python package "foursquare" (no need to send HTTP requests directly with library `requests`)
- **categoryId** for bars and nightlife needs to be found in the [foursquare API documentation](https://developer.foursquare.com/docs/api-reference/venues/search/)

In [ ]:
# 2022-10-02 21:38 Overall approach:
    # Used Foursquare API to find bars near 6 of the accidents (first 3 and last 3)
    # Found mean number of bars according to Injury type
    # Conclusion: Unable to determine if there's a signficant relationship between accident severity and number of nearby 
    # bars  due to small sample size.

In [3]:
# 2022-10-02 10:39: Original function. No longer needed.
def get_venues(coordinates,category='13003',radius=5):
    """Parameters:
    - Coordinates: (string) Latitude and longitude, separated by a comma.
    - Category: (string, optional) Category of place. If blank default is 13003 for 
    'Dining and Drinking > Bar'
    - Radius: (integer, optional) Radius of search results.
    """
    radius = int(radius)
    urlAppend = f'?ll={coordinates}&radius={radius}&categories={category}'
    search_URL = "https://api.foursquare.com/v3/places/search"+urlAppend
    
    return search_URL

get_venues('48,17')

'https://api.foursquare.com/v3/places/search?ll=48,17&radius=5&categories=13003'

In [134]:
# 2022-10-02 10:40 Custom function 
def get_venues(coordinates,urlEP='https://api.foursquare.com/v3/places/search',category='13003',radius=5000):
    """Parameters:
    - Coordinates: (string) Latitude and longitude, separated by a comma.
    - Term: (string, optional) Search term to use. Default is 'coworking'.
    - urlEP: (string, optional) Root for API endpoint URL. Default for Foursquare Places.
    - Category: (string, optional) Category of place. 
        If blank default is 13003 for 'Dining and Drinking > Bar'
            - 11128 for 'Business and Professional Services > Office > Coworking Space'
            - 13003 for 'Dining and Drinking > Bar'. 
        - In Yelp: Ethiopian restaurants: 'ethiopian'
    - Radius: (integer, optional) Radius of search results.
    
    Returns:
    - URL for API call (string)
    """
    urlEP=str(urlEP)
    radius = int(radius)
    category=str(category)

    if urlEP.find('api.yelp') > 0:
        coordinates = str.split(coordinates,sep=',')
        urlAppend = f'?latitude={coordinates[0]}&longitude={coordinates[1]}&radius={radius}&term={category}' # yelp
    else:
        urlAppend = f'?ll={coordinates}&radius={radius}&categories={category}' # Default is for foursquare
    request_URL=urlEP+urlAppend
    return request_URL

In [104]:
# Add searchURL to dataframe
# data['foursquareURL'] = str(get_venues(data['ll']))
data.head(5)
# 2022-10-02 20:09: Turns out that search URL becomes a weird data type when retrieved from the DF, so I removed it.

,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,ll
0,902363382,2015,1,5,Weekday,0.0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356"
1,902364268,2015,1,6,Weekday,1500.0,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848"
2,902364412,2015,1,6,Weekend,2300.0,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006"
3,902364551,2015,1,7,Weekend,900.0,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635"
4,902364615,2015,1,7,Weekend,1100.0,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482"


In [32]:
# See what injury types there are
data['Injury Type'].value_counts()

No injury/unknown     37467
Non-incapacitating    10427
Incapacitating         1003
Fatal                   108
Name: Injury Type, dtype: int64

In [182]:
#set the keys
apiKey = os.environ["API_FOURSQUARE"]
# apiKey
# foursquare_secret = os.environ[""]

In [8]:

url = "https://api.foursquare.com/v3/places/search"

headers = {
    "Accept": "application/json",
    "Authorization": apiKey
}

# response = requests.request("GET", url, headers=headers)

{"results":[{"fsq_id":"4c44da1af0bdd13aee2ccacc","categories":[{"id":16023,"name":"Lake","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/parks_outdoors/lake_","suffix":".png"}}],"chains":[],"distance":1145,"geocodes":{"main":{"latitude":49.163382,"longitude":-123.997793}},"link":"/v3/places/4c44da1af0bdd13aee2ccacc","location":{"country":"CA","cross_street":"","formatted_address":"Nanaimo BC","locality":"Nanaimo","region":"BC"},"name":"Westwood Lake","related_places":{},"timezone":"America/Vancouver"},{"fsq_id":"4b5516daf964a520cada27e3","categories":[{"id":10006,"name":"Bowling Alley","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/arts_entertainment/bowling_","suffix":".png"}}],"chains":[],"distance":901,"geocodes":{"main":{"latitude":49.168469,"longitude":-123.986414},"roof":{"latitude":49.168469,"longitude":-123.986414}},"link":"/v3/places/4b5516daf964a520cada27e3","location":{"address":"171 Calder Rd","country":"CA","cross_street":"","formatted_address":"171 Cald

In [67]:
# play around with indexing
data.iloc[0,:]

Master Record Number                                            902363382
Year                                                                 2015
Month                                                                   1
Day                                                                     5
Weekend?                                                          Weekday
Hour                                                                  0.0
Collision Type                                                      2-Car
Injury Type                                             No injury/unknown
Primary Factor                      OTHER (DRIVER) - EXPLAIN IN NARRATIVE
Reported_Location                                              1ST & FESS
Latitude                                                        39.159207
Longitude                                                      -86.525874
ll                                               39.15920668,-86.52587356
foursquareURL           https://api.fo

In [137]:
# Make the GET request for Foursquare for first few accidents in the dataset. 

responses = dict()
for crash in range(3):
    url = get_venues(data['ll'][crash])
    print(url)
    responses[str(crash)] = requests.get(url, headers=headers)
    # responses[str(data['Master Record Number'][crash])] = 'test' # 2022-10-02 20:11: Originally planned to index responses
        # by Master Record Number, but then realized using the index is easier.
responses

https://api.foursquare.com/v3/places/search?ll=39.15920668,-86.52587356&radius=5000&categories=13003
https://api.foursquare.com/v3/places/search?ll=39.16144,-86.534848&radius=5000&categories=13003
https://api.foursquare.com/v3/places/search?ll=39.14978027,-86.56889006&radius=5000&categories=13003


{'0': <Response [200]>, '1': <Response [200]>, '2': <Response [200]>}

In [154]:
# Loop through API responses to create all the dataframes at once. 
dfs = dict()
for key, value in responses.items():
    # print(key, value.json())
    json = responses[key].json()
    dfs[key] = json_normalize(json, record_path='results')


In [160]:
print(dfs['0'].shape)
dfs['0'].head(3)


(10, 23)


,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,...,location.country,location.cross_street,location.dma,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,location.po_box,related_places.children
0,507c2800e4b0b5e9215c9444,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],1151,/v3/places/507c2800e4b0b5e9215c9444,The Tap,America/Indiana/Indianapolis,39.166841,-86.535183,39.166841,...,US,,Indianapolis,"101 N College Ave, Bloomington, IN 47404",Bloomington,[Arlington],47404,IN,NaN,NaN
1,5249fca911d20ce66163e75a,"[{'id': 13009, 'name': 'Cocktail Bar', 'icon':...",[],966,/v3/places/5249fca911d20ce66163e75a,Cardinal Spirits,America/Indiana/Indianapolis,39.156283,-86.536519,39.156283,...,US,,Indianapolis,"922 S Morton St, Bloomington, IN 47403",Bloomington,NaN,47403,IN,P.O. Box 2282,NaN
2,4ae33784f964a520cd9121e3,"[{'id': 13025, 'name': 'Wine Bar', 'icon': {'p...",[],1869,/v3/places/4ae33784f964a520cd9121e3,Upland Brewing Company Brew Pub,America/Indiana/Indianapolis,39.173503,-86.537557,39.173503,...,US,at Rogers St.,Indianapolis,"350 W 11th St (at Rogers St.), Bloomington, IN...",Bloomington,[Arlington],47404,IN,NaN,NaN


In [204]:
# Play around with indexing the bottom of the dataframe
data['Master Record Number'].iloc[-1]

900046599

In [206]:
# Make the GET request for Foursquare for last 3 accidents in the dataset. 
for crash in range(-4,-1):
    # print(crash)
    url = get_venues(data['ll'].iloc[crash]) # Need to use .iloc to index the last rows of the dataframe
    print(url)
    responses[str(crash)] = requests.get(url, headers=headers)
responses

https://api.foursquare.com/v3/places/search?ll=39.002752,-86.463856&radius=5000&categories=13003
https://api.foursquare.com/v3/places/search?ll=38.99232624,-86.53725171&radius=5000&categories=13003
https://api.foursquare.com/v3/places/search?ll=38.99152,-86.448784&radius=5000&categories=13003


{'0': <Response [200]>,
 '1': <Response [200]>,
 '2': <Response [200]>,
 '-4': <Response [200]>,
 '-3': <Response [200]>,
 '-2': <Response [200]>}

In [208]:
# Loop through API responses to create all the dataframes for new responses. 
for key, value in responses.items():
    # print(key, value.json())
    json = responses[key].json()
    dfs[key] = json_normalize(json, record_path='results')


{'0':                      fsq_id  \
 0  507c2800e4b0b5e9215c9444   
 1  5249fca911d20ce66163e75a   
 2  4ae33784f964a520cd9121e3   
 3  5b07718bd3cce8002cd23f9b   
 4  5064d1ebe4b0675fa422bb50   
 5  535ae26c498e51ccd1e78ace   
 6  5390b62d498e1eb46529a20a   
 7  4ae3d332f964a5202f9921e3   
 8  53ea4a71498e76cdafe2ee7c   
 9  4aebd299f964a520e3c421e3   
 
                                           categories chains  distance  \
 0  [{'id': 13003, 'name': 'Bar', 'icon': {'prefix...     []      1151   
 1  [{'id': 13009, 'name': 'Cocktail Bar', 'icon':...     []       966   
 2  [{'id': 13025, 'name': 'Wine Bar', 'icon': {'p...     []      1869   
 3  [{'id': 13003, 'name': 'Bar', 'icon': {'prefix...     []      1393   
 4  [{'id': 10032, 'name': 'Night Club', 'icon': {...     []       998   
 5  [{'id': 13003, 'name': 'Bar', 'icon': {'prefix...     []       936   
 6  [{'id': 13002, 'name': 'Bakery', 'icon': {'pre...     []      1258   
 7  [{'id': 13003, 'name': 'Bar', 'icon': {'prefi

In [272]:
# Save the JSON data so it can be referred to without repeating API requests
# Test out with first API request.
responses['0'].json()

with open(f'foursquareBar0','w') as json_file:
    json.dump(responses['0'].json(), json_file)

In [274]:
for key, value in responses.items():
    # print(key, value.json())
    # print(f'foursquareBar{key}')
    with open(f'foursquareBar{key}.json','w') as json_file:
        json.dump(responses[key].json(), json_file)

In [213]:
dfs['-4']

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,...,location.census_block,location.country,location.cross_street,location.dma,location.formatted_address,location.locality,location.postcode,location.region,related_places.parent.fsq_id,related_places.parent.name
0,4ff8c5d7e4b08a4c73420d18,"[{'id': 13013, 'name': 'Hotel Bar', 'icon': {'...",[],2148,/v3/places/4ff8c5d7e4b08a4c73420d18,Wind Jammer Grill,America/Indiana/Indianapolis,39.016595,-86.481348,39.016595,...,181050015022001,US,,Indianapolis,"9301 S Fairfax Rd, Bloomington, IN 47401",Bloomington,47401,IN,NaN,NaN
1,4b512b1cf964a520ac4527e3,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],4196,/v3/places/4b512b1cf964a520ac4527e3,Fairfax Inn,America/Indiana/Indianapolis,39.034080,-86.490740,39.034080,...,181050015013035,US,Lake Monroe,Indianapolis,"8660 S Fairfax Rd (Lake Monroe), Bloomington, ...",Bloomington,47401,IN,NaN,NaN
2,003b4c9d48d84685a8f4f7e2,"[{'id': 13013, 'name': 'Hotel Bar', 'icon': {'...",[],2164,/v3/places/003b4c9d48d84685a8f4f7e2,Windjammer,NaN,39.016573,-86.481511,39.016573,...,181050015022001,US,NaN,Indianapolis,"9301 S Fairfax Rd, Bloomington, IN 47401",Bloomington,47401,IN,4b6c3fc1f964a520b02b2ce3,Fourwinds Resort & Marina


In [232]:
# Pull the following data:
    # 'Injury Type' from accident data
    # Number of bars from each accident

injury_nBar = [] # This will be a list of dictionaries of relevant info per accident.

for accidentID, df in dfs.items():
    # print(accidentID, df.shape[0]) # test out the iteration
    # print(accidentID, df.shape[0], data['Injury Type'].iloc[int(accidentID)]) # test out the iteration
    injury_nBar.append({
        'accidentID': str(accidentID),
        'Injury Type': data['Injury Type'].iloc[int(accidentID)],
        'nBars': df.shape[0]
    })

,accidentID,Injury Type,nBars
0,0,No injury/unknown,10
1,1,No injury/unknown,10
2,2,Non-incapacitating,10
3,-4,Non-incapacitating,3
4,-3,No injury/unknown,2
5,-2,Non-incapacitating,2


In [239]:
# Get the average number of bars per injury type.
df_injury_nBar = pd.DataFrame(injury_nBar)
df_injury_nBar.groupby('Injury Type').agg({'nBars': 'mean'})

# Larger sample size required to infer statistical significance.

,nBars
Injury Type,
No injury/unknown,7.333333
Non-incapacitating,5.000000


# World Weather Online API

World Weather Online API is [here](https://www.worldweatheronline.com/developer/api/historical-weather-api.aspx)

1. Sign up for FREE api key if you haven't done that before (it's free for **30 days**).
2. For each crush, get the weather for the location and date.
3. Find a relationship between the weather and severity of the crash.

Hints:

* pull weather only for smaller sample of crashes (250 or so) due to API limits
* for sending HTTP requests check out "requests" library [here](http://docs.python-requests.org/en/master/)


In [193]:
import requests
import time
api_key = os.environ["api_worldweather"]